In [1]:
import pandas as pd
from tqdm import tqdm
import vectorbt as vbt
from vietcap import VietCapClient
from fialda import FialdaClient

### TÍN HIỆU MUA HÔM NAY
***Giả sử, ở phiên giao dịch gần nhất, NĐT chỉ quan tâm đến các mã CP có:***
- `ev` (*vốn hóa*) >= 1e+12 (*1 nghìn tỷ đồng*)
- `averageVolume3Month` (*trung bình cộng KLGD 3 tháng*) >= 2e+5 (*200.000 CP/phiên*)
- `SMA10` (*trung bình trượt 10 phiên của giá đóng cửa*) **cắt lên trên** `SMA20` (*trung bình trượt 20 phiên của giá đóng cửa*) 
    - <code>SMA10<sub>t-1</sub></code>  < <code>SMA20<sub>t-1</sub></code>
    - `SMA10` > `SMA20`
- `SMA20` (*trung bình trượt 20 phiên của giá đóng cửa*) > `SMA50` (*trung bình trượt 50 phiên của giá đóng cửa*)
- `SMA50` (*trung bình trượt 50 phiên của giá đóng cửa*) > `SMA100` (*trung bình trượt 100 phiên của giá đóng cửa*)
- `MACD` > `MACD_Signal`

In [2]:
fialda = FialdaClient()
payload = """
    {
        "faFilter": {
            "AvgVol3M": {
                "min": 200000,
                "max": null
            },
            "MarketCap": {
                "min": 1000000000000,
                "max": null
            },
            "TotalDealVol": {
                "min": null,
                "max": null
            },
            "LastPrice": {
                "min": null,
                "max": null
            }
        },
        "taFilter": null,
        "booleanFilter": {
            "AvailableForFASearching": true
        },
        "pageNumber": 1,
        "pageSize": 10000,
        "exchanges": [
            "HSX",
            "HNX",
            "UPCOM"
        ],
        "icbCodes": null,
        "sortColumn": "Symbol",
        "isDesc": false,
        "fAFilterSub": null,
        "faKeys": [
            "SMA10_VUOT_SMA20_Daily",
            "SMA10_>_SMA50_Daily",
            "MACD_>=_MACDSignal_Daily",
            "SMA20_>_SMA50_Daily",
            "SMA50_>_SMA100_Daily"
        ],
        "wlOrPId": null,
        "tradingTime": null
    }
"""
df_buy_signal = fialda.get_stock_data_by_filter(payload)
# số mã CP có tín hiệu MUA
print("Số mã CP có tín hiệu MUA:", df_buy_signal.shape[0])
df_buy_signal.drop(columns=["CompTypeCode","FSCreationTime","Year","Quarter"], errors="ignore")

Số mã CP có tín hiệu MUA: 6


,Symbol,AvgVol3M,MarketCap,TotalDealVol,LastPrice,SMA10_Daily,SMA20_Daily,SMA50_Daily,MACD_Daily,MACDSignal_Daily,SMA100_Daily
0,GIL,9.832667e+05,2.113201e+12,2766500.0,20.80,19.675,19.6300,18.8450,0.3812,0.2993,17.4495
1,GVR,4.813411e+06,1.306000e+14,9515500.0,32.65,30.930,30.7550,30.0000,0.5399,0.3122,28.8645
2,HPX,3.829832e+06,1.715511e+12,7029100.0,5.64,5.350,5.3225,4.7946,0.2070,0.1883,4.4827
3,MSN,7.900267e+06,1.306103e+14,24211400.0,85.90,77.390,76.8150,73.3240,2.2375,1.3026,67.6940
4,VCS,2.124303e+05,8.160000e+12,351200.0,51.00,50.250,50.1400,49.0465,0.5486,0.4293,48.0173
5,VOS,3.006308e+06,2.373000e+12,6033300.0,16.95,15.630,15.5450,14.9920,0.3287,0.2099,14.4935


### LỊCH SỬ GIAO DỊCH
***Lấy dữ liệu lịch sử giao dịch của các mã CP hôm nay có tín hiệu MUA***

In [3]:
vcsc = VietCapClient()
# lấy dữ liệu lịch sử từ 2016
from_date = "2016-01-01"
df_historical_price_bs = pd.DataFrame()
for ticker in tqdm(df_buy_signal["Symbol"]):
    # lấy dữ liệu lịch sử từ vcsc
    data = vcsc.get_historical_price(ticker, from_date=from_date)
    # chỉ append nếu có dữ liệu
    if len(data) > 0:
        # chuyển dữ liệu dạng list thành bảng
        df = pd.DataFrame(data)
        # chuyển cột `tradingDate` từ dạng timestamp thành datetime
        df["tradingDate"] = pd.to_datetime(df["tradingDate"], unit="ms")
        # append dữ liệu
        df_historical_price_bs = pd.concat([df_historical_price_bs, df], ignore_index=True)

100%|██████████| 6/6 [00:01<00:00,  4.86it/s]


In [4]:
# dữ liệu giao dịch có nhiều cột
df_historical_price_bs.iloc[0]

tradingDate                 2025-08-12 00:00:00
ticker                                      GIL
openPrice                                 20450
highestPrice                              20900
lowestPrice                               19650
closePrice                                20900
openPriceAdjusted                       20450.0
highestPriceAdjusted                    20900.0
lowestPriceAdjusted                     19650.0
closePriceAdjusted                      20900.0
totalMatchVolume                        1603494
totalMatchValue                     32219140350
totalDealVolume                               0
totalDealValue                                0
unMatchedBuyTradeVolume               1045562.0
unMatchedSellTradeVolume              1438926.0
totalVolume                             1603494
totalValue                          32219140350
totalBuyTrade                            1249.0
totalBuyTradeVolume                   2649056.0
totalSellTrade                          

In [5]:
# rút ngắn dữ liệu cần thiết cho BACKTESTING gồm giá `OHLC` (điều chỉnh) và KL khớp lệnh `totalMatchVolume`
# tính thêm các chỉ số cần thiết để BACKTESTING
df_historical_price_bs_bt = (
    df_historical_price_bs
    .loc[:,["tradingDate","ticker","openPriceAdjusted","highestPriceAdjusted","lowestPriceAdjusted","closePriceAdjusted","totalMatchVolume"]]
    .rename(columns=lambda x: x.replace("PriceAdjusted", ""))
    .sort_values(["ticker","tradingDate"])
    .reset_index(drop=True)
    .assign(
        SMA10  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(10).mean()),
        SMA20  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(20).mean()),
        SMA50  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(50).mean()),
        SMA100 = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.rolling(100).mean()),
        EMA12  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.ewm(span=12, adjust=False).mean()),
        EMA26  = lambda df: df.groupby("ticker")["close"].transform(lambda x: x.ewm(span=26, adjust=False).mean()),
        MACD   = lambda df: df["EMA12"] - df["EMA26"],
        MACDSignal = lambda df: df.groupby("ticker")["MACD"].transform(lambda x: x.ewm(span=9, adjust=False).mean())
    )
)
df_historical_price_bs_bt.head()

,tradingDate,ticker,open,highest,lowest,close,totalMatchVolume,SMA10,SMA20,SMA50,SMA100,EMA12,EMA26,MACD,MACDSignal
0,2016-01-07,GIL,16849.178170,16897.456617,16366.393695,16656.064380,38210,NaN,NaN,NaN,NaN,16656.064380,16656.064380,0.000000,0.000000
1,2016-01-08,GIL,16656.064380,16656.064380,16366.393695,16366.393695,32020,NaN,NaN,NaN,NaN,16611.499659,16634.607292,-23.107633,-4.621527
2,2016-01-11,GIL,16704.342827,16704.342827,16366.393695,16366.393695,5720,NaN,NaN,NaN,NaN,16573.791049,16614.739618,-40.948569,-11.886935
3,2016-01-12,GIL,16414.672142,16414.672142,16366.393695,16366.393695,2830,NaN,NaN,NaN,NaN,16541.883764,16596.343624,-54.459860,-20.401520
4,2016-01-13,GIL,16414.672142,16511.229037,16366.393695,16414.672142,4380,NaN,NaN,NaN,NaN,16522.312745,16582.886477,-60.573732,-28.435962


In [6]:
# xây dựng logic tín hiệu MUA
## đặt biến
sma10       = df_historical_price_bs_bt["SMA10"]
sma20       = df_historical_price_bs_bt["SMA20"]
sma50       = df_historical_price_bs_bt["SMA50"]
sma100      = df_historical_price_bs_bt["SMA100"]
macd        = df_historical_price_bs_bt["MACD"]
macd_signal = df_historical_price_bs_bt["MACDSignal"]
sma10_lag1  = df_historical_price_bs_bt.groupby("ticker")["SMA10"].shift(1)
sma20_lag1  = df_historical_price_bs_bt.groupby("ticker")["SMA20"].shift(1)
## SMA10 cắt lên SMA20
sma10_crossover_sma20 = (sma10 > sma20) & (sma10_lag1 <= sma20_lag1)
## SMA20 > SMA50
sma20_over_sma_50 = sma20 > sma50
## SMA50 > SMA100
sma50_over_sma_100 = sma50 > sma100
## MACD > MACDsignal
macd_over_macdsignal = macd > macd_signal
## tín hiệu MUA
buy_signal = sma10_crossover_sma20 & sma20_over_sma_50 & sma50_over_sma_100 & macd_over_macdsignal

In [7]:
# xây dựng logic tín hiệu BÁN
## SMA10 cắt xuống SMA20
sell_signal = (sma10 < sma20) & (sma10_lag1 >= sma20_lag1)

In [8]:
## gán ngược tín hiệu MUA/BÁN vào bảng
df_historical_price_bs_bt["buySignal"]  = buy_signal
df_historical_price_bs_bt["sellSignal"] = sell_signal
df_historical_price_bs_bt.head()

,tradingDate,ticker,open,highest,lowest,close,totalMatchVolume,SMA10,SMA20,SMA50,SMA100,EMA12,EMA26,MACD,MACDSignal,buySignal,sellSignal
0,2016-01-07,GIL,16849.178170,16897.456617,16366.393695,16656.064380,38210,NaN,NaN,NaN,NaN,16656.064380,16656.064380,0.000000,0.000000,False,False
1,2016-01-08,GIL,16656.064380,16656.064380,16366.393695,16366.393695,32020,NaN,NaN,NaN,NaN,16611.499659,16634.607292,-23.107633,-4.621527,False,False
2,2016-01-11,GIL,16704.342827,16704.342827,16366.393695,16366.393695,5720,NaN,NaN,NaN,NaN,16573.791049,16614.739618,-40.948569,-11.886935,False,False
3,2016-01-12,GIL,16414.672142,16414.672142,16366.393695,16366.393695,2830,NaN,NaN,NaN,NaN,16541.883764,16596.343624,-54.459860,-20.401520,False,False
4,2016-01-13,GIL,16414.672142,16511.229037,16366.393695,16414.672142,4380,NaN,NaN,NaN,NaN,16522.312745,16582.886477,-60.573732,-28.435962,False,False


### BACKTESTING

In [9]:
backtesting_results = {}
for ticker, df in tqdm(df_historical_price_bs_bt.groupby("ticker")):
    # dùng cột ngày để làm index cho bảng
    df.set_index("tradingDate", inplace=True)
    # loại bỏ 100 phiên đầu tiên vì tính `SMA100`
    df = df.iloc[99:]
    # chạy backtesting bằng `vectorbt`
    pf = vbt.Portfolio.from_signals(
        close=df["close"],
        entries=df["buySignal"],
        exits=df["sellSignal"],
        size=1.0,       # mua toàn bộ vốn cho mỗi lệnh
        fees=0.001,     # phí giao dịch 0.1%
        freq="1D"
    )
    # lưu kết quả backtesting
    backtesting_results[ticker] = pf

100%|██████████| 6/6 [00:03<00:00,  1.57it/s]


In [14]:
# chuyển kết quả backtesting thành bảng
df_backtesting = pd.concat({ticker: pf.stats() for ticker, pf in backtesting_results.items()}, axis=1)
# chuyển dataframe thành dạng markdown table
# print(df_backtesting.map(lambda x: round(x,2) if isinstance(x, float) else x).to_markdown())
df_backtesting.map(lambda x: round(x,2) if isinstance(x, float) else x)

,GIL,GVR,HPX,MSN,VCS,VOS
Start,2016-06-06 00:00:00,2018-08-10 00:00:00,2018-12-11 00:00:00,2016-06-06 00:00:00,2016-06-06 00:00:00,2016-06-06 00:00:00
End,2025-08-12 00:00:00,2025-08-12 00:00:00,2025-08-12 00:00:00,2025-08-12 00:00:00,2025-08-12 00:00:00,2025-08-12 00:00:00
Period,2298 days 00:00:00,1743 days 00:00:00,1663 days 00:00:00,2298 days 00:00:00,2299 days 00:00:00,2298 days 00:00:00
Start Value,100.0,100.0,100.0,100.0,100.0,100.0
End Value,159.26,235.79,95.29,94.73,103.03,90.81
Total Return [%],59.26,135.79,-4.71,-5.27,3.03,-9.19
Benchmark Return [%],-21.29,330.68,-67.73,95.82,91.84,654.55
Max Gross Exposure [%],100.0,100.0,100.0,100.0,100.0,100.0
Total Fees Paid,2.13,2.9,1.13,2.69,1.23,0.7
Max Drawdown [%],43.08,15.93,24.41,44.46,22.18,39.29


### KẾT QUẢ PHÂN TÍCH 
***Nhờ chatGPT đọc kết quả backtesting :)))***

***1. Hiệu suất tổng thể***
- GIL: Lãi 59.26%, nhưng Max Drawdown khá cao (43%), Sharpe Ratio 0.48 → lợi nhuận có nhưng biến động mạnh, rủi ro cao.
- GVR: Lãi 135.78%, Win Rate 77.77%, Sharpe Ratio 0.96 → tương đối tốt, rủi ro thấp (DD ~15%).
- HPX: Lỗ -4.71%, Win Rate chỉ 16.67%, Sharpe Ratio âm → gần như chiến lược không phù hợp.
- MSN: Lỗ -5.27%, Sharpe Ratio âm, Drawdown 44% → rủi ro rất cao, hiệu suất kém.
- VCS: Lãi nhẹ 3.02%, Sharpe Ratio dương nhưng thấp → lợi nhuận không đáng kể.
- VOS: Lỗ -9.18%, Drawdown 39%, Sharpe Ratio âm → kém hiệu quả.

***2. Rủi ro và ổn định***
- Max Drawdown (mức sụt giảm vốn lớn nhất): HPX (24%), MSN (44%), GIL (43%), VOS (39%) → các mã này có rủi ro drawdown lớn.
- Calmar Ratio (lợi nhuận năm hóa / max drawdown): GVR nổi bật (0.91) → lợi nhuận tốt so với rủi ro.

***3. Khả năng thắng***
- Win Rate: GVR (77.77%), GIL (57%), MSN (50%) → cao hơn 50% là tốt, nhưng cần kết hợp với Profit Factor để chắc hơn.
- Profit Factor: GVR (14.46), GIL (1.52) → trên 1 là chiến lược có lợi thế, càng cao càng tốt.

**Kết luận nhanh**
- GVR là mã duy nhất trong nhóm vừa có lợi nhuận cao, vừa rủi ro thấp, vừa chỉ số Sharpe & Calmar khá tốt → ứng viên đáng theo dõi.
- GIL có lợi nhuận ổn nhưng rủi ro khá cao, cần quản lý vốn tốt.
- HPX, MSN, VOS cho thấy chiến lược không phù hợp trong giai đoạn backtest.
- VCS gần hòa vốn, không đủ hấp dẫn.

In [11]:
# biểu diễn biểu đồ vốn của GVR (mã đáng theo dõi nhất)
pf = backtesting_results["GVR"]
pf.plot().show()

In [12]:
# lưu thành image
pf.plot().write_image("backtesting_plot.png")